In [ ]:
from vctube import VCtube
playlist_name = 'ecomony'
playlist_url = 'https://youtu.be/XtVE-9ywfDc?si=uXntZ9n72wI8sKfy'
lang = 'ko'
vc = VCtube(playlist_name, playlist_url, lang)

In [ ]:
import yt_dlp as youtube_dl
import os


def check_korean_subtitles_and_down(video_url, download_path):
    ydl_opts = {
        'quiet': True,
        'subtitleslangs': 'all',
        'writesubtitles': True,
        'writeautomaticsub': True,
        'skip_download': True,
        'format': 'best',
    }
    
    download_path = os.path.join('datasets', download_path, "wavs/" + '%(id)s.%(ext)s')

    # youtube_dl options
    ydl_opts_down = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192'
        }],
        'postprocessors_args': [
            '-ar', '21000'
        ],
        'prefer_ffmpeg': True,
        'keepvideo': False,
        'outtmpl': download_path,  # 다운로드 경로 설정
        'ignoreerrors': True
    }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        try:
            result = ydl.extract_info(video_url, download=False)
            if result:
                subtitles = result.get('subtitles', {})
                auto_subtitles = result.get('automatic_captions', {})
                # 한국어 자막 또는 자동 생성 자막 확인
                has_korean_subs = 'ko' in subtitles or 'ko' in auto_subtitles
                
                try:
                    with youtube_dl.YoutubeDL(ydl_opts_down) as ydl_down:
                        ydl_down.download([video_url])
                except Exception as e:
                    print('error', e)
                
        except Exception as e:
            print(f"에러: {e}")
            return False
    

    

# 예제 사용법:
video_url = 'https://www.youtube.com/watch?v=example'
print("한국어 자막 사용 가능:" , check_korean_subtitles(video_url))

In [4]:
vc.download_audio()

[youtube] Extracting URL: https://youtu.be/XtVE-9ywfDc?si=uXntZ9n72wI8sKfy
[youtube] XtVE-9ywfDc: Downloading webpage
[youtube] XtVE-9ywfDc: Downloading ios player API JSON
[youtube] XtVE-9ywfDc: Downloading mweb player API JSON
[youtube] XtVE-9ywfDc: Downloading m3u8 information
[info] XtVE-9ywfDc: Downloading 1 format(s): 251
[download] Destination: ./datasets/GM_meeting/wavs/XtVE-9ywfDc.webm
[download] 100% of   10.23MiB in 00:00:05 at 1.81MiB/s     
[ExtractAudio] Destination: ./datasets/GM_meeting/wavs/XtVE-9ywfDc.wav
Deleting original file ./datasets/GM_meeting/wavs/XtVE-9ywfDc.webm (pass -k to keep)


In [6]:
vc.download_captions()

100%|██████████| 1/1 [00:10<00:00, 10.79s/it]

 [*] Make directories : ./datasets/GM_meeting/text
GM_meeting channel was finished


In [7]:
vc.audio_split()

Split with caption: 100%|██████████| 1/1 [00:00<00:00,  2.19it/s]


In [10]:
from datasets import Dataset, Audio
import pandas as pd

df = pd.read_csv('/home/gue707/PromptStyler/test/datasets/GM_meeting/text/subtitle.csv')
df

,Unnamed: 0,id,text,start,duration,name
0,0,6p_bQjqE9Hg,음악론 하나님께서 보는대로 성 말,6.040,3.330,./datasets/GM_meeting/wavs/6p_bQjqE9Hg.0000.wav
1,1,6p_bQjqE9Hg,되었으므로 지금부터 제 24,9.370,3.179,./datasets/GM_meeting/wavs/6p_bQjqE9Hg.0001.wav
2,2,6p_bQjqE9Hg,미사의 개념을 사랑합니다,12.549,2.491,./datasets/GM_meeting/wavs/6p_bQjqE9Hg.0002.wav
3,3,6p_bQjqE9Hg,에,15.040,1.980,./datasets/GM_meeting/wavs/6p_bQjqE9Hg.0003.wav
4,4,6p_bQjqE9Hg,블레이드 1 그 성장 소변기 그를,17.020,2.849,./datasets/GM_meeting/wavs/6p_bQjqE9Hg.0004.wav
...,...,...,...,...,...,...
473,473,6p_bQjqE9Hg,질이라는 것은 좀 솔직 많이 안,1132.220,1.920,./datasets/GM_meeting/wavs/6p_bQjqE9Hg.0473.wav
474,474,6p_bQjqE9Hg,됩니다,1134.140,1.620,./datasets/GM_meeting/wavs/6p_bQjqE9Hg.0474.wav
475,475,6p_bQjqE9Hg,그러니까 그러면 확하게 젖어서 타결을,1135.760,2.520,./datasets/GM_meeting/wavs/6p_bQjqE9Hg.0475.wav
476,476,6p_bQjqE9Hg,확실하게 되요 좀 저것들 더우,1138.280,2.430,./datasets/GM_meeting/wavs/6p_bQjqE9Hg.0476.wav


In [11]:
ds = Dataset.from_pandas(df).remove_columns(['Unnamed: 0', 'id', 'start', 'duration']).cast_column("name", Audio(sampling_rate=16000))
ds

Dataset({
    features: ['text', 'name'],
    num_rows: 478
})

In [4]:
ds[0]

{'text': 'good evening everybody',
 'name': {'path': './datasets/Obama_Speech/wavs/bps3m4eFTuE.0000.wav',
  'array': array([3.50772636e-04, 8.91358592e-04, 5.68822143e-04, ...,
         9.37006436e-04, 3.32841300e-04, 2.92773475e-05]),
  'sampling_rate': 16000}}

In [5]:
ds.push_to_hub('Obama_Speech')

Uploading the dataset shards: 100%|██████████| 1/1 [00:17<00:00, 17.70s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/imTak/Obama_Speech/commit/b53ccc586b987154e0f52f49474605bcd8cd246f', commit_message='Upload dataset', commit_description='', oid='b53ccc586b987154e0f52f49474605bcd8cd246f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/imTak/Obama_Speech', endpoint='https://huggingface.co', repo_type='dataset', repo_id='imTak/Obama_Speech'), pr_revision=None, pr_num=None)

# Pyannote Audio

In [ ]:
from pyannote.audio import Pipeline
pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1")

# send pipeline to GPU (when available)
import torch
pipeline.to(torch.device("cuda"))

# apply pretrained pipeline
import time

start_time = time.time()
diarization = pipeline("XtVE-9ywfDc.wav")
end_time = time.time()

elapsed = end_time - start_time

print(elapsed)
# print the result
for turn, _, speaker in diarization.itertracks(yield_label=True):
    print(f"start={turn.start:.1f}s stop={turn.end:.1f}s speaker_{speaker}")

/home/gue707/.conda/envs/test/lib/python3.10/site-packages/pyannote/audio/models/blocks/pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1823.)
  std = sequences.std(dim=-1, correction=1)


121.86165475845337
start=4.1s stop=4.6s speaker_SPEAKER_05
start=13.5s stop=14.0s speaker_SPEAKER_05
start=14.0s stop=44.6s speaker_SPEAKER_01
start=37.5s stop=37.9s speaker_SPEAKER_00
start=46.0s stop=51.5s speaker_SPEAKER_01
start=51.8s stop=56.7s speaker_SPEAKER_01
start=56.9s stop=57.8s speaker_SPEAKER_01
start=58.3s stop=61.1s speaker_SPEAKER_01
start=61.8s stop=74.9s speaker_SPEAKER_01
start=77.8s stop=105.9s speaker_SPEAKER_01
start=106.1s stop=108.8s speaker_SPEAKER_01
start=109.1s stop=110.7s speaker_SPEAKER_01
start=111.0s stop=112.2s speaker_SPEAKER_01
start=112.5s stop=115.4s speaker_SPEAKER_01
start=115.8s stop=127.0s speaker_SPEAKER_01
start=127.8s stop=130.9s speaker_SPEAKER_01
start=130.9s stop=142.0s speaker_SPEAKER_01
start=142.6s stop=143.6s speaker_SPEAKER_01
start=144.3s stop=145.2s speaker_SPEAKER_01
start=146.1s stop=148.2s speaker_SPEAKER_01
start=148.5s stop=150.9s speaker_SPEAKER_01
start=151.7s stop=157.8s speaker_SPEAKER_01
start=160.6s stop=165.1s speaker_S

In [15]:
from pydub import AudioSegment
import os
from tqdm import tqdm

audio = AudioSegment.from_file("XtVE-9ywfDc.wav")

output_dir = "output_speakers/test" 
os.makedirs(output_dir, exist_ok=True)

for turn, _, speaker in tqdm(diarization.itertracks(yield_label=True)):
    if (turn.end - turn.start) >= 1.0:  # 1초 이상 지속되는 발화
        start_ms = int(turn.start * 1000)  # 시작 시간을 밀리초로 변환
        end_ms = int(turn.end * 1000)      # 종료 시간을 밀리초로 변환
        segment = audio[start_ms:end_ms]   # 해당 부분 잘라내기
        segment.export(f"{output_dir}/{start_ms}_{end_ms}_speaker_{speaker}.wav", format="wav")

173it [00:00, 734.04it/s]


In [3]:
from faster_whisper import WhisperModel
import os

Nicknames = ['민지', '해린', '정곤', '카리나', '유나', '설윤', '고윤정', '하니']

model = WhisperModel("imTak/faster-whisper_Korean_L3turbo", device="cpu")

wav_dir = "output_speakers/test"
file_list = sorted(os.listdir(wav_dir))
file_list = [os.path.join(wav_dir, file) for file in file_list]


In [4]:
from tqdm import tqdm

Nicknames = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

with open('RESULT.txt', 'wt', encoding='utf-8') as f:

    for file in tqdm(file_list):
        
        nick = Nicknames[int(file.split('_')[-1][:-4])]
        
        segments, info = model.transcribe(file, language='ko')
        
        for segment in segments:
            m, s = divmod(segment.start, 60)
            time_str = f"{int(m):02d}:{int(s):02d}" if m > 0 else f"{int(s):02d}"
            f.write(f"[{time_str}] {nick}: {segment.text}\n")
            print(f"{nick}: {segment.text}")
    
    f.close()

  0%|          | 0/103 [00:00<?, ?it/s]

  1%|          | 1/103 [00:09<15:46,  9.28s/it]

B: 올림픽을 해서 진짜 다니는 것 같은 게


  2%|▏         | 2/103 [00:20<17:53, 10.63s/it]

B: 좀 막이 없잖아


...

B: 그럭지 않은데 오미지 말겨요


 99%|█████████▉| 102/103 [17:59<00:10, 10.31s/it]

B: 이게 문제다라는 지금 공통적인 문제제기는 됐으니까 밥 먹고 어떤 전략으로 갈까에 대한 얘기를 이어서 해 봅시다


100%|██████████| 103/103 [18:09<00:00, 10.58s/it]

B: 내가 무한도전 시즌투를 못하는 이유에 대한 기사가 할 수 있었어요
B: 근데 거기 있는 댓글들이 절대 못한다 무한도전 시즌투는 그런 의견이 엄청 지지적인가 못하는 이유가 요즘 세상에 무한우전하면 용만 먹고 그 시즌원의 그런 추억들만 다 훼손될 거다 분명히 무한우전 시즌투가 그렇게 될 거다라는 얘기들을 봤는데 나는 요즘


In [5]:
from tqdm import tqdm

Nicknames = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

segments, info = model.transcribe('XtVE-9ywfDc.wav', language='ko')

with open('RESULT2.txt', 'wt', encoding='utf-8') as f:
    
    for segment in segments:
        m, s = divmod(int(segment.start), 60)
        time_str = f"{int(m)}:{int(s)}" if m > 0 else f"{int(s)}"
        f.write(f"[{time_str}] {nick}: {segment.text}\n")
        print(f"[{time_str}] {nick}: {segment.text}")
    
    f.close()

B: 몇 자막 제공 및 자막 제공 및 자막 제공 및 광고를 포기한다
B:  제도는 제도다
B: 그리고 불편한 것 같아 사실 무시하는 것도 하나의 방법일 수도 있고 이렇게 불편력을 꺼져 보면 이렇게 하는 것도 방법일 수도 있고 한데 나는 그거를 직접적으로 해봤으면 어떨까 싶어
B: 그런데 거기 있는 댓글들에 절대 못한다 무안전 시즌2는 그런 의견이 엄청 질적이며 못하는 이유가 요즘 세상에 무안전 하면 용만 먹고 그 시즌1의 그런 추억들만 다 훼손될 뿐일까 분명히 무안전 시즌2가 그렇게 될 거다라는 얘기들을 봤는데 나는 요즘
B: 올림픽니다 나 올림픽이다
B:  이 논의 주택으로 미쳤다
B: 부지적인 생활이긴 한데 아직까지 우리나라에서 나오는 게 지금 메시지에서 일곱은 다 젖에서 나가면 불편한 문제들이 그래서 그걸 할 수 있는 방법은 번화를 좀 바꾸게 하는 게 힘들 것 같다
B: 따라서 웃으면서 합시다
B: 그 나는 채널에서 아무리 잘되는 순간이더라도 지금 문제가 뭔지 약간 솔직하게 좀 사람들로 소통하고 싶거든요 아 요즘 주유수 개그치 않나는가 이러면서 얘기하고 싶단 말이야 우리의 걱정 고민 이슈들을 그걸 얘기하면 지금은 약점이 돼 뭐 예를 들면 요즘 주유수가 안 나와가지고 뭐 그렇게 얘기하면 너가 이러니까 주유수가 안 나오지부터 시작이 된다
B: 몬�엘�다
B: 몬�여다
B: 몬�여다
B: 몬�여다
B: 몬�여
B: 몬�여다
B: 우리 직업이 직업으로 인정받기 위한 일 열 과정인데 첫번째는 단짝 떠 그 다음 과정이 이 사람들이 이렇게 뜨기 위한 주의 모습들 그런 노력들이 그다음에 한 번씩 조명이 되면서 그다음에 직업으로 인정되게 하는 건데 지금 사실 유튜브는 어떤 상황인 것 같으면 어떤 노력을 하는지 아직 사람들은 하나도 몰라 유튜브들이 어떻게 영상을 제작하고 어떤 이렇게 고민을 하는지 하나도 모르는 상태고 뭐냐면
B:  별로 전문성도 없어 보인 애들이 엄청 유명해졌어 그런데 제일 조명 아닌 건 유튜버들 플렉스야 돈 여자 플렉스 외에 유튜�라는 직업에 대해서 재

In [ ]:
import os
from openai import OpenAI


client = OpenAI(
    api_key=open('API_KEY.txt', 'r').read()
)

chat_completion = client.chat.completions.create(
    messages=[
        { 'role': "system", 'content': "You are a assistant to clarify the meeting in a meeting script." },
        {
            "role": "user",
            "content": 
                f"""회의 대본 : {open('RESULT2.txt', 'r').read()}
                    회의 대본은 회의 음성을 텍스트로 변환한 거야. 다음 지침을 따라줘. 
                    1. 한국어로 요약해줘.         
                    2. 회의 대본에서 각 주제별로 요약하고 결론을 자세히 알려줘. 화자는 언급하지 않아도 돼.
                    3. 회의 대본에서 더 조사하려는 내용이 있었으면 해당 내용의 검색 키워드를 알려줘.
                    4. 회의 내용 중 다음 회의 일정을 언급했다면 미팅 예정일과 시간, 장소를 알려줘.
                    5. 아래 출력 형식으로 Markdown code를 작성해줘.
                    6. 출력 형식 : '
                    (##)topic summary: 
                        1. 주제 1 
                            * summry : 회의 내용 요약
                            * result : 회의 결론
                            * search : (이텔릭체)추가로 조사할 내용에 대한 검색 문장
                        2. 주제 2 
                            * summary : 회의 내용 요약
                            * result : 회의 결론
                            * search : (이텔릭체)추가로 조사할 내용에 대한 검색 문장
                        ...
                        
                    *next meeting*: (날짜(월/일 or 이번주 금요일), 시간(오전/오후 n시 or 미정), 장소(없을 경우 미정) or 없음
                    '
                    """
        }
    ],
    model="gpt-3.5-turbo",
    temperature=0.1,
)
print(chat_completion.choices[0].message.content)



```
## AI 어플리케이션 개발 주제 요약:
    1. 인공지능을 활용한 애플리케이션 아이디어 발상
        * summary: AI를 활용한 앱으로 사용자의 불편을 해소하는 방향으로 아이디어 발상
        * result: AI를 활용한 회의 어시스턴트 서비스 개발을 결정
        * search: AI를 활용한 회의 어시스턴트 서비스 구현 방법

    2. AI 회의 어시스턴트 서비스 기능 및 차별화 아이디어
        * summary: 회의 내용 녹음, 텍스트 변환, 캘린더 관리, 자료 조사 지원 등의 기능 제안
        * result: AI를 활용한 회의 어시스턴트 서비스의 차별화된 기능 도출
        * search: AI를 활용한 회의 어시스턴트 서비스의 기능 구현 방법

    3. 다음 회의 일정 및 과제
        * summary: 다음 회의는 29일에 예정되며, UI/UX 디자인, 기획, 백엔드 플로우 구현 등의 과제를 분담
        * result: 다음 회의는 29일에 학교 카페에서 진행
        * search: AI 어플리케이션 개발에 필요한 한국어 STT 모델 파인튜닝 방법

*next meeting*: 29일, 미정, 학교 카페
```


# Simple Diarizer

In [47]:
import soundfile as sf
import matplotlib.pyplot as plt

from simple_diarizer.diarizer import Diarizer
from simple_diarizer.utils import combined_waveplot

diar = Diarizer(
                  embed_model='xvec', # 'xvec' and 'ecapa' supported
                  cluster_method='sc' # 'ahc' and 'sc' supported
               )

WAV_FILE = 'XtVE-9ywfDc.wav'

NUM_SPEAKERS=5

segments = diar.diarize(WAV_FILE, num_speakers=NUM_SPEAKERS)

signal, fs = sf.read(WAV_FILE)
# 저장할 디렉토리 생성 (없을 경우)
output_directory = 'output_segments'
os.makedirs(output_directory, exist_ok=True)

for seg in segments:
    
    if seg['end'] - seg['start'] < 1.0:
        continue
    
    start = seg["start_sample"]
    end = seg["end_sample"]
    speech = signal[start:end]

    filename = f"{int(seg['start'])}_{int(seg['end'])}_speaker_{seg['label']}.wav"
    file_path = os.path.join(output_directory, filename)

    sf.write(file_path, speech, fs)

    print(f"Speaker {seg['label']} ({seg['start']}s - {seg['end']}s) saved to {file_path}")
        

Using cache found in /home/gue707/.cache/torch/hub/snakers4_silero-vad_master
INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch embedding_model.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch mean_var_norm_emb.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch classifier.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch label_encoder.txt: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: embedding_model, mean_var_norm_

Converting audio file to single channel WAV using ffmpeg...


ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 11.2.0 (Anaconda gcc)
  configuration: --prefix=/croot/ffmpeg_1722024521393/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_ --cc=/croot/ffmpeg_1722024521393/_build_env/bin/x86_64-conda-linux-gnu-cc --ar=/croot/ffmpeg_1722024521393/_build_env/bin/x86_64-conda-linux-gnu-ar --nm=/croot/ffmpeg_1722024521393/_build_env/bin/x86_64-conda-linux-gnu-nm --ranlib=/croot/ffmpeg_1722024521393/_build_env/bin/x86_64-conda-linux-gnu-ranlib --strip=/croot/ffmpeg_1722024521393/_build_env/bin/x86_64-conda-linux-gnu-strip --disable-doc --enable-swresample --enable-swscale --enable-openssl --enable-libxml2 --enable-libtheora --enable-demuxer=dash --enable-postproc --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfont

Running VAD...
Splitting by silence found 312 utterances
Extracting embeddings...


Utterances: 100%|██████████| 312/312 [00:01<00:00, 222.91it/s]
/home/gue707/.conda/envs/test/lib/python3.10/site-packages/sklearn/manifold/_spectral_embedding.py:310: UserWarning: Array is not symmetric, and will be converted to symmetric by average with its transpose.
  adjacency = check_symmetric(adjacency)


Clustering to 5 speakers...
Cleaning up output...
Done!
Speaker 3 (13.73s - 15.8379375s) saved to output_segments/13_15_speaker_3.wav
Speaker 3 (16.29s - 20.0939375s) saved to output_segments/16_20_speaker_3.wav
Speaker 3 (21.154s - 23.9339375s) saved to output_segments/21_23_speaker_3.wav
Speaker 3 (24.514s - 27.1019375s) saved to output_segments/24_27_speaker_3.wav
Speaker 3 (27.426s - 29.0539375s) saved to output_segments/27_29_speaker_3.wav
Speaker 3 (29.282s - 30.3339375s) saved to output_segments/29_30_speaker_3.wav
Speaker 3 (31.106s - 34.1419375s) saved to output_segments/31_34_speaker_3.wav
Speaker 0 (35.458s - 36.7019375s) saved to output_segments/35_36_speaker_0.wav
Speaker 0 (37.218s - 39.2939375s) saved to output_segments/37_39_speaker_0.wav
Speaker 3 (39.874s - 41.4379375s) saved to output_segments/39_41_speaker_3.wav
Speaker 3 (42.21s - 44.6379375s) saved to output_segments/42_44_speaker_3.wav
Speaker 3 (46.082s - 47.6779375s) saved to output_segments/46_47_speaker_3.wav

In [48]:
from tqdm import tqdm
from faster_whisper import WhisperModel
import os

model = WhisperModel("imTak/faster-whisper_Korean_L3turbo", device="cpu")

wav_dir = "output_segments/"
file_list = sorted(os.listdir(wav_dir))
file_list = [os.path.join(wav_dir, file) for file in file_list]

Nicknames = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

with open('RESULT_cpu.txt', 'wt', encoding='utf-8') as f:

    for file in tqdm(file_list):
        
        nick = Nicknames[int(file.split('_')[-1][:-4])]
        
        segments, info = model.transcribe(file, language='ko')
        
        for segment in segments:
            m, s = divmod(segment.start, 60)
            time_str = f"{int(m):02d}:{int(s):02d}" if m > 0 else f"{int(s):02d}"
            f.write(f"[{time_str}] {nick}: {segment.text}\n")
            print(f"{nick}: {segment.text}")
    
    f.close()

  0%|          | 1/239 [00:07<29:23,  7.41s/it]

D: 오오호


INFO:faster_whisper:Processing audio with duration 00:00.532
  1%|          | 2/239 [00:19<40:30, 10.25s/it]

A: 저씨의 주인공을 받았을 때까지 했다


INFO:faster_whisper:Processing audio with duration 00:00.415
  1%|▏         | 3/239 [00:30<42:00, 10.68s/it]

...

D: 지금이 불닿병이 불닿병이 불닿병이라고 말했다


INFO:faster_whisper:Processing audio with duration 00:00.500
100%|█████████▉| 238/239 [42:31<00:13, 13.78s/it]INFO:faster_whisper:Processing audio with duration 00:03.290


A: 현재


100%|██████████| 239/239 [42:41<00:00, 10.72s/it]

D: 전날 밤샘이 폭동구 예습이관 이 조금


In [ ]:
import os
from openai import OpenAI


client = OpenAI(
    api_key=open('API_KEY.txt', 'r').read()
)

chat_completion = client.chat.completions.create(
    messages=[
        { 'role': "system", 'content': "You are a assistant to clarify the meeting in a meeting script." },
        {
            "role": "user",
            "content": 
                f"""회의 대본 : {open('RESULT_cpu.txt', 'r').read()}
                회의 대본은 회의 음성을 텍스트로 변환한 거야. 다음 지침을 따라줘. 
                    1. 한국어로 요약해줘.         
                    2. 회의 대본에서 각 주제별로 요약하고 결론을 자세히 알려줘. 화자는 언급하지 않아도 돼.
                    3. 회의 대본에서 더 조사하려는 내용이 있었으면 해당 내용의 검색 키워드를 알려줘.
                    4. 회의 내용 중 다음 회의 일정을 언급했다면 미팅 예정일과 시간, 장소를 알려줘.
                    5. 아래 출력 형식으로 Markdown code를 작성해줘.
                    6. 출력 형식 : '
                    (제목2)topic summary: 
                        1. 주제 1 
                            * summry : 회의 내용 요약
                            * result : 회의 결론
                            * search : (이텔릭체)추가로 조사할 내용에 대한 검색 문장
                        2. 주제 2 
                            * summary : 회의 내용 요약
                            * result : 회의 결론
                            * search : (이텔릭체)추가로 조사할 내용에 대한 검색 문장
                    (인용)next meeting: (날짜(월/일 or 이번주 금요일), 시간(오전/오후 n시 or 미정), 장소(없을 경우 미정) or 없음
                    '
                    """
        }
    ],
    model="gpt-3.5-turbo",
    temperature=0.2,
)
print(chat_completion.choices[0].message.content)



```
## topic summary: 
   1. 홍보 및 일본의 자리에 대한 의혹
       * summary: 회의 참여자들이 홍보 및 일본의 자리에 대한 의혹에 대해 토론함
       * result: 홍보 및 의혹에 대한 논의가 계속될 것으로 보임
       * search: 홍보 및 의혹 관련 사례 연구

   2. 저씨의 주장과 의견 제시
       * summary: 저씨가 주장하고 의견을 제시하는 내용에 대한 토론
       * result: 저씨의 주장이 회의에 영향을 미칠 것으로 예상됨
       * search: 저씨의 주장에 대한 추가 정보

   3. 홍빛과 홍병에 대한 논의
       * summary: 홍빛과 홍병에 대한 토론 및 결론 도출
       * result: 홍빛과 홍병에 대한 조치가 필요함
       * search: 홍빛 및 홍병 관련 전략

## next meeting: 이번주 금요일 오후 2시
```
